# Installing and setting up environment

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install pyyaml==5.1
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 KB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp39-cp39-linux_x86_64.whl size=44089 sha256=940e711528a120b98f1a815b0d711ee83287a0c6b9f4c99d7dc4ce1d0fc2a7c9
  Stored in directory: /root/.cache/pip/wheels/68/be/8f/b6c454cd264e0b349b47f8ee00755511f277618af9e5dae20d
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flax 0.6.7 requires PyYAML>=5.4.1, but you have pyyaml 5.1 which is incompatible.
dask 2022.12.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# After this step it will ask you to restart the runtime, please do it.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61429 sha256=ab48b19e194d885fe483eaa263abe3da49f914848d84b09a13922d7c6b6308b0
  Stored in directory: /root/.cache/pip/wheels/83/42/02/66178d16e5c44dc26d3099318

In [ ]:
import torch
assert torch.__version__.startswith("1.8") 
import torchvision
import cv2

In [ ]:
import os
import numpy as np
import json
import random
import matplotlib.pyplot as plt
%matplotlib inline

from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog

# Data

This cell below creates our data dictionary

In [ ]:
def get_data_dicts(directory, classes):
    dataset_dicts = []
    idx = 0
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["height"] = img_anns["imageHeight"]
        record["width"] = img_anns["imageWidth"]
        record["image_id"] = idx
        idx = idx+1
        
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS, #BoxMode.XYWH_ABS or BoxMode.XYXY_ABS
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

This two cells below shouldn't both be run instead we run either the cell or panel depending on which one you would like to train

In [ ]:
classes = ['solar_panel'] #this would be the name of the class you have

data_path = '/content/drive/MyDrive/Test Folder SD1 Multiple Poly/' #this would be the folder of where your data is stored

for d in ["train", "test"]:
    DatasetCatalog.register("category_" + d, lambda d=d: get_data_dicts(data_path+d, classes))
    MetadataCatalog.get("category_" + d).set(thing_classes=classes)

microcontroller_metadata = MetadataCatalog.get("category_")

In [ ]:
classes = ['solar_cell']

data_path = '/content/drive/MyDrive/cell data PV warped modules/'

for d in ["train", "test"]:
    DatasetCatalog.register("category_" + d, lambda d=d: get_data_dicts(data_path+d, classes))
    MetadataCatalog.get("category_" + d).set(thing_classes=classes)

microcontroller_metadata = MetadataCatalog.get("category_")

In [ ]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer

# Training model

Here as you can see we use category_train from either the panel or cells to train the model

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("category_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

[03/28 16:03:46 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:03, 53.8MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [ ]:
trainer.train()

[03/28 16:04:06 d2.engine.train_loop]: Starting training from iteration 0
[03/28 16:04:20 d2.utils.events]:  eta: 0:00:52  iter: 19  total_loss: 2.63  loss_cls: 0.6802  loss_box_reg: 0.7658  loss_mask: 0.6803  loss_rpn_cls: 0.4232  loss_rpn_loc: 0.1441  time: 0.6928  data_time: 0.0484  lr: 4.7703e-05  max_mem: 3144M
[03/28 16:04:34 d2.utils.events]:  eta: 0:00:39  iter: 39  total_loss: 1.872  loss_cls: 0.5228  loss_box_reg: 0.6939  loss_mask: 0.5408  loss_rpn_cls: 0.04246  loss_rpn_loc: 0.09459  time: 0.6749  data_time: 0.0168  lr: 9.7653e-05  max_mem: 3206M
[03/28 16:04:48 d2.utils.events]:  eta: 0:00:26  iter: 59  total_loss: 1.473  loss_cls: 0.3991  loss_box_reg: 0.6279  loss_mask: 0.3513  loss_rpn_cls: 0.01613  loss_rpn_loc: 0.07155  time: 0.6819  data_time: 0.0199  lr: 0.0001476  max_mem: 3206M
[03/28 16:05:02 d2.utils.events]:  eta: 0:00:13  iter: 79  total_loss: 1.179  loss_cls: 0.2563  loss_box_reg: 0.548  loss_mask: 0.2694  loss_rpn_cls: 0.005495  loss_rpn_loc: 0.07114  time: 

# Saving model weight and making predictor

The weight of the model is save to model_final.pth in the output folder

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.DATASETS.TEST = ("data_test", )
predictor = DefaultPredictor(cfg) #this will be our predictor model
     

In [ ]:
from detectron2.checkpoint import DetectionCheckpointer

However it is also save as model since we use the checkpointer for our project but the one above should be the same

In [ ]:
checkpointer = DetectionCheckpointer(trainer.model , save_dir = cfg.OUTPUT_DIR)
checkpointer.save("model")

# Making inferences on model

Here we are creating our evaluator and validation loader

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("category_test", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "category_test")

[03/28 16:05:19 d2.evaluation.coco_evaluation]: Trying to convert 'category_test' to COCO format ...
[03/28 16:05:19 d2.data.datasets.coco]: Converting annotations of dataset 'category_test' to COCO format ...)
[03/28 16:05:52 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[03/28 16:05:52 d2.data.datasets.coco]: Conversion finished, #images: 36, #annotations: 2143
[03/28 16:05:52 d2.data.datasets.coco]: Caching COCO format annotations at './output/category_test_coco_format.json' ...
[03/28 16:05:53 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
| solar_cell | 2143         |
|            |              |
[03/28 16:05:53 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[03/28 16:05:53 d2.data.common]: Serializing 36 elements to byte tensors and concatenating them all ...
[03/28 1

This cell below will make inferences on the predictor model with the evaluator and val_loader

In [ ]:
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

[03/28 16:05:53 d2.evaluation.evaluator]: Start inference on 36 batches
[03/28 16:06:03 d2.evaluation.evaluator]: Inference done 11/36. Dataloading: 0.0417 s/iter. Inference: 0.1422 s/iter. Eval: 0.4374 s/iter. Total: 0.6213 s/iter. ETA=0:00:15
[03/28 16:06:09 d2.evaluation.evaluator]: Inference done 20/36. Dataloading: 0.0489 s/iter. Inference: 0.1445 s/iter. Eval: 0.4438 s/iter. Total: 0.6373 s/iter. ETA=0:00:10
[03/28 16:06:14 d2.evaluation.evaluator]: Inference done 24/36. Dataloading: 0.0391 s/iter. Inference: 0.1554 s/iter. Eval: 0.5835 s/iter. Total: 0.7783 s/iter. ETA=0:00:09
[03/28 16:06:21 d2.evaluation.evaluator]: Inference done 30/36. Dataloading: 0.0322 s/iter. Inference: 0.1609 s/iter. Eval: 0.6553 s/iter. Total: 0.8488 s/iter. ETA=0:00:05
[03/28 16:06:25 d2.evaluation.evaluator]: Total inference time: 0:00:25.820750 (0.832927 s / iter per device, on 1 devices)
[03/28 16:06:25 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:04 (0.160581 s / iter per devi

AP is APm when we have one class so we should look at AP since APm is average precision of all classes and we are also using  the bounding box not the segmentation